In [1]:
from Bio import SeqIO, SeqFeature
import pandas as pd
import numpy as np
import sys
import os
import typing as t
from itertools import chain
from collections import Counter
import skbio
from Bio import Phylo
from skbio import TreeNode

In [2]:
os.chdir("/home/is6/work/cas_thesis/16s_extraction/final_round/thesis_results")

In [3]:
#Read tree that is made from filtered and new pfam domains
with open('/home/is6/work/cas_thesis/16s_extraction/final_round/thesis_results/bacteria_tree.nwk') as buffer:
    tree = skbio.io.read(buffer, format="newick", into=skbio.tree.TreeNode)

In [4]:
#rooting the tree at midpoint
tree = tree.root_at_midpoint()
tree.write(open('rooted_tree.nwk', 'w'))

<_io.TextIOWrapper name='rooted_tree.nwk' mode='w' encoding='UTF-8'>

In [5]:
#reading cluster file
cluster_assignments = pd.read_csv("/home/ilia/projects/misc/sedreh/domains_pfam/assigned_components_annotated_filt.tsv", sep='\t')
cluster_assignments.head()

,id,component,genus_renamed
0,GCF_000006605.1,1,Corynebacterium
1,GCF_000006985.1,8,Chlorobium
2,GCF_000007085.1,3,Caldanaerobacter
3,GCF_000007425.1,5,Streptococcus
4,GCF_000007465.2,10,Streptococcus


In [7]:
#Make statistics table with node_id and clusters
#label rooted tree and 
final_dir = '/home/is6/work/cas_thesis/16s_extraction/final_round/thesis_results/'

genus_to_clusters = {
    genus: grp['component']
    for genus, grp in cluster_assignments.groupby('genus_renamed')
}

node_components: t.List[dict] = [] #creating empty list
with open(final_dir + 'rooted_tree.nwk') as buffer: #opening our rooted tree as buffer
    tree = skbio.io.read(buffer, format="newick", into=skbio.tree.TreeNode)  #loading tree from the buffer
    # label internal nodes
    tree.assign_ids(),   
    for node in tree.traverse(): #traversing tree to assign internal node labeles
        if not node.is_tip():   #ignore the tip nodes
             node.name = f'n{node.id}' 
             components = Counter(chain.from_iterable(
                 genus_to_clusters[tip.name] for tip in node.tips() if tip.name in genus_to_clusters 
             ))  
             node_components.append({'node_id': node.name, **components})
                
pd.DataFrame.from_records(node_components).to_csv(final_dir + 'statistics.csv')
with open(final_dir + 'bacteria_final_tree_labelled.nwk', 'w') as buffer:
    tree.write(buffer, format="newick")